In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


data = []


url = "https://www.airbnb.co.in/s/Mumbai-Maharashtra-IN/homes?dynamic_product_ids%5B%5D=1475969398243265317&omni_page_id=36021&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRym9mVDI5zsRrqh0xGAazB4&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2026-01-01&monthly_length=3&monthly_end_date=2026-04-01&price_filter_input_type=2&channel=EXPLORE&ne_lat=19.215219370696452&ne_lng=73.04314947924897&sw_lat=18.979625424515177&sw_lng=72.81890371112459&zoom=11.587508105686474&zoom_level=11&search_by_map=true&query=Mumbai%20Maharashtra%20IN&price_filter_num_nights=5&pagination_search=true&federated_search_session_id=b1346757-21e8-47cd-8394-fa33a5865641&cursor=eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjowLCJ2ZXJzaW9uIjoxfQ%3D%3D"

r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

while True:

    cards = soup.find_all("div", {"itemprop": "itemListElement"})

    for card in cards:
        # Name
        name = card.find("div", {"class": "t1jojoys atm_g3_1kw7nm4 atm_ks_15vqwwr atm_sq_1l2sidv atm_9s_cj1kg8 atm_6w_1e54zos atm_fy_1vgr820 atm_7l_hfv0h6 atm_cs_1mexzig atm_w4_1eetg7c atm_ks_zryt35__1rgatj2 dir dir-ltr"})
        name = name.text if name else "NA"

        # Price
        price = card.find("span", {"class": "umg93v9 atm_7l_mhi6a6 atm_cs_80qqwn atm_rd_14k51in atm_cs_l3jtxx__1v156lz dir dir-ltr"})
        price = price.text if price else "NA"

        # Reviews
        review = card.find("span", {"class": "r4a59j5"})
        if review:
            # Extract only the numeric part like 4.85

            match = re.search(r"\d+\.\d+", review.text)
            review = match.group() if match else "NA"
        else:
            review = "NA"


        # Description
        desc = card.find("span", {"class": "t6mzqp7 atm_g3_1kw7nm4 atm_ks_15vqwwr atm_sq_1l2sidv atm_9s_cj1kg8 atm_6w_1e54zos atm_fy_kb7nvz atm_7l_xeyu1p atm_am_qk3dho atm_ks_zryt35__1rgatj2 dir dir-ltr"})
        desc = desc.text if desc else "NA"

        data.append([name, price, review, desc])

    # pagination
    next_btn = soup.find("a", {"aria-label": "Next"})
    if next_btn is None:
        print("Reached last page")
        break

    next_url = "https://www.airbnb.co.in" + next_btn.get("href")
    r = requests.get(next_url)
    soup = BeautifulSoup(r.text, "lxml")

df = pd.DataFrame(data, columns=["Name", "Price", "Reviews", "Description"])
print(df)


Reached last page
                              Name    Price Reviews  \
0                Apartment in Juhu  ₹23,058    4.76   
1                   Room in Mumbai  ₹18,259    4.88   
2              Room in Bandra West       NA    4.84   
3           Flat in Santacruz East  ₹45,648    4.96   
4                Room in Khar West   ₹8,519    4.83   
..                             ...      ...     ...   
265    Apartment in Santacruz West  ₹43,999    4.93   
266            Flat in Bandra West       NA    4.92   
267      Tiny home in Andheri East   ₹8,034      NA   
268  Place to stay in Andheri West   ₹8,445    4.75   
269            Flat in Bandra West  ₹35,162    4.78   

                                           Description  
0                  Bliss - Entire Studio in Vile Parle  
1                           Bedroom at Delphi Bungalow  
2    Cozy room with park view in Penthouse, Bandra ...  
3       Beautiful Studio Apartment Near Mumbai Airport  
4                 Nani Ka Ghar - Moo

In [ ]:
df.to_csv("AirbnbMumbai.csv")